In [15]:
import pandas as pd

침수횟수

In [16]:
침수지역=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\침수지역2022.csv")
침수횟수=pd.DataFrame(침수지역[["GU_NAME","ADM_NM"]].value_counts()).reset_index()
강우량=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\강우량.xlsx")
구단위=pd.DataFrame(강우량.groupby(["구청명","자료수집 시각"])["10분우량"].mean())
구단위.reset_index(inplace=True)
#구단위
구단위총합=구단위.groupby(['구청명'])['10분우량'].agg(["sum","count"]).reset_index()
구단위총합.rename(columns={"sum":'총우량',"count":'측정횟수(10분단위)'},inplace=True)
구단위총합["면적당시우량"]=구단위총합["총우량"]/구단위총합["측정횟수(10분단위)"]*6
구단위총합
구단위총합
인구밀도=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\서울시_행정동별_인구밀도_2023.csv")
인구밀도
구비율_행정동면적=pd.DataFrame(인구밀도.groupby(["구"])["면적 (k㎡)"].sum()).reset_index()
구비율_행정동면적=pd.merge(인구밀도,구비율_행정동면적,left_on="구",right_on='구')
구비율_행정동면적.rename(columns={'면적 (k㎡)_x':'동면적(k㎡)',"면적 (k㎡)_y":"구면적(k㎡)"},inplace=True)
구비율_행정동면적['동/구면적']=구비율_행정동면적["동면적(k㎡)"]/구비율_행정동면적['구면적(k㎡)']
merge_data=pd.merge(구비율_행정동면적,구단위총합,left_on="구",right_on="구청명")
# merge_data['동단위우량']=merge_data["면적당시우량"]*merge_data["동면적(k㎡)"]
# merge_data["동단위시우량"]=merge_data["면적당시우량"]*merge_data["동/구면적"]
merge_data.drop(["index",'행정구역코드','구면적(k㎡)','구청명','총우량','측정횟수(10분단위)'],axis=1,inplace=True)
# merge_data.rename(columns={"계 (명)":'인구수'},inplace=True)
merge_data.rename(columns={"구":"GU_NAME","행정동":'ADM_NM'},inplace=True)
merge_data
merge_data=pd.merge(merge_data,침수횟수,how="left").rename(columns={'count':"침수횟수"})
merge_data.fillna(0,inplace=True) #침수횟수 nan->0

In [17]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0
...,...,...,...,...,...,...,...,...
421,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0
422,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0
423,강동구,강일동,2.26,33082,14638.053100,0.091907,8.79600,0.0
424,강동구,상일1동,1.82,39435,21667.582420,0.074014,8.79600,0.0


In [18]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0
...,...,...,...,...,...,...,...,...
421,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0
422,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0
423,강동구,강일동,2.26,33082,14638.053100,0.091907,8.79600,0.0
424,강동구,상일1동,1.82,39435,21667.582420,0.074014,8.79600,0.0


경사도

In [19]:
경사도=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\decisiontree\경사도.xlsx")
경사도=pd.DataFrame(경사도.groupby(["GU_NAME","ADM_NM"])["경사도"].mean()).reset_index()
merge_data=pd.merge(merge_data,경사도,how='inner')

In [20]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667
...,...,...,...,...,...,...,...,...,...
410,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143
411,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154
412,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786
413,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909


In [21]:
merge_data["ADM_NM"].value_counts()

ADM_NM
신사동     2
사직동     1
독산4동    1
양평2동    1
양평1동    1
       ..
송천동     1
송중동     1
미아동     1
삼양동     1
강일동     1
Name: count, Length: 414, dtype: int64

배수

In [24]:
배수=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\배수등급양호비율.csv")
배수["ADM_NM"].value_counts()

ADM_NM
가락1동    1
송천동     1
오륜동     1
오류2동    1
오류1동    1
       ..
방학3동    1
방학2동    1
방학1동    1
방이2동    1
흑석동     1
Name: count, Length: 417, dtype: int64

In [ ]:
merge_data=pd.merge(merge_data,배수[["GU_NAME","ADM_NM","양호비율"]],how="inner")
merge_data

녹지

In [ ]:
녹지=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\area.csv",encoding='euc-kr')
녹지

,Column1,ADM_NM,AREA(k㎡),ROAD_BT,LENGTH,ROAD_AREA,SHAPE_AREA,도로_건물_면적,도로_건물_면적_km²,GREEN_AREA,행정동별_녹지면적(km²),도로_건물_면적_비율,녹지_면적_비율
0,0,사직동,1.23,3.0,23837.242370,0.071512,306229.52,0.377741,0.377741,475.463437,0.475463,0.307107,0.386556
1,1,삼청동,1.49,3.0,15490.784850,0.046472,157766.19,0.204239,0.204239,956.136070,0.956136,0.137073,0.641702
2,2,부암동,2.27,3.0,29312.630200,0.087938,245752.96,0.333691,0.333691,511.686673,0.511687,0.147000,0.225413
3,3,평창동,8.87,5.0,53320.458890,0.266602,511877.61,0.778480,0.778480,1459.884861,1.459885,0.087765,0.164587
4,4,무악동,0.36,3.0,5646.249268,0.016939,44051.59,0.060990,0.060990,166.167613,0.166168,0.169418,0.461577
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,409,성내3동,0.71,4.0,17694.730520,0.070779,240222.74,0.311002,0.311002,4.093324,0.004093,0.438031,0.005765
410,410,길동,1.61,5.0,33203.116040,0.166016,410807.46,0.576823,0.576823,94.996411,0.094996,0.358275,0.059004
411,411,둔촌1동,0.92,24.5,4415.158442,0.108171,102907.97,0.211079,0.211079,41.522737,0.041523,0.229434,0.045133
412,412,둔촌2동,1.56,5.0,21017.148080,0.105086,245598.21,0.350684,0.350684,229.326026,0.229326,0.224797,0.147004


In [ ]:
merge_data=pd.merge(merge_data,녹지,how='inner')
merge_data.drop(['Column1', 'AREA(k㎡)', 'ROAD_BT', 'LENGTH',
       'ROAD_AREA', 'SHAPE_AREA', '도로_건물_면적', '도로_건물_면적_km²', 'GREEN_AREA','행정동별_녹지면적(km²)'],axis=1,inplace=True)

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,0.307107,0.386556
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,0.137073,0.641702
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,0.147000,0.225413
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,0.087765,0.164587
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,0.169418,0.461577
...,...,...,...,...,...,...,...,...,...,...,...,...
403,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,0.438031,0.005765
404,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,0.358275,0.059004
405,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,0.229434,0.045133
406,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,0.224797,0.147004


하천까지의거리

In [ ]:
하천=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\하천까지의_거리.csv")
하천['하천거리(km)']=하천["HubDist"]/1000
# merge_data=pd.merge(merge_data,하천[["GU_NAME","ADM_NM",'하천거리(km)']],how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,0.307107,0.386556
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,0.137073,0.641702
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,0.147000,0.225413
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,0.087765,0.164587
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,0.169418,0.461577
...,...,...,...,...,...,...,...,...,...,...,...,...
403,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,0.438031,0.005765
404,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,0.358275,0.059004
405,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,0.229434,0.045133
406,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,0.224797,0.147004


빗물펌프장까지의거리

In [ ]:
# 빗물펌프장_거리=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\거리계산.xlsx")
# 빗물펌프장_거리.rename(columns=dict(zip(빗물펌프장_거리.columns,["ADM_NM","펌프장","빗물펌프장까지의거리(m)"])),inplace=True)
# 빗물펌프장_정보=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\빗물펌프장_전처리.xlsx")
# 펌프장거리정보=pd.merge(빗물펌프장_거리,빗물펌프장_정보)
# 펌프장거리정보.drop(["Unnamed: 0","위치"],axis=1,inplace=True)
# 펌프장거리정보
# 펌프장=pd.DataFrame(펌프장거리정보.groupby(["GU_NAME","ADM_NM"])["빗물펌프장까지의거리(m)"].min()).reset_index()
# merge_data=pd.merge(merge_data,펌프장,how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,0.307107,0.386556
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,0.137073,0.641702
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,0.147000,0.225413
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,0.087765,0.164587
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,0.169418,0.461577
...,...,...,...,...,...,...,...,...,...,...,...,...
403,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,0.438031,0.005765
404,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,0.358275,0.059004
405,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,0.229434,0.045133
406,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,0.224797,0.147004


제방암거수문개수

In [ ]:
제방=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\decisiontree\제방암거수문개수.csv",encoding="euc-kr")
제방

,GU_NAME,ADM_NM,제방,수문,암거
0,강남구,개포1동,65,4,58
1,강남구,개포2동,30,11,95
2,강남구,개포4동,47,3,48
3,강남구,논현1동,0,0,64
4,강남구,논현2동,0,0,93
...,...,...,...,...,...
417,중랑구,상봉2동,1,0,16
418,중랑구,신내1동,56,8,243
419,중랑구,신내2동,3,0,26
420,중랑구,중화1동,4,0,12


In [ ]:
merge_data=pd.merge(merge_data,제방,how='inner')
merge_data

관공서개수

In [ ]:
관공서=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\인프라_행정동별_관공서합계(경찰서,병원,소방서).xlsx")
merge_data=pd.merge(merge_data,관공서.rename(columns={'동별':"ADM_NM"})[["ADM_NM","병원","경찰","소방서"]],how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율,제방,수문,암거,병원,경찰,소방서
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,0.307107,0.386556,3,0,95,75,3,0
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,0.137073,0.641702,0,0,43,11,1,0
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,0.147000,0.225413,40,3,40,4,1,1
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,0.087765,0.164587,83,20,171,15,1,0
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,0.169418,0.461577,0,0,10,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,0.438031,0.005765,0,0,17,80,0,0
403,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,0.358275,0.059004,4,0,21,103,0,1
404,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,0.229434,0.045133,0,0,9,1,0,0
405,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,0.224797,0.147004,19,1,68,28,2,0


노후건물

In [ ]:
노후건물=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\개수지롱.csv")
merge_data=pd.merge(merge_data,노후건물[["GU_NAME","ADM_NM","NUMPOINTS"]].rename(columns={'NUMPOINTS':"노후건물개수"}),how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율,제방,수문,암거,병원,경찰,소방서,노후건물개수
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,0.307107,0.386556,3,0,95,75,3,0,485
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,0.137073,0.641702,0,0,43,11,1,0,344
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,0.147000,0.225413,40,3,40,4,1,1,972
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,0.087765,0.164587,83,20,171,15,1,0,1331
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,0.169418,0.461577,0,0,10,6,0,0,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,0.438031,0.005765,0,0,17,80,0,0,425
403,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,0.358275,0.059004,4,0,21,103,0,1,892
404,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,0.229434,0.045133,0,0,9,1,0,0,11
405,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,0.224797,0.147004,19,1,68,28,2,0,574


소득가구비율

In [ ]:
# 소득가구=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\decisiontree\반출_행정동_소득가구_수정2차.xlsx")
# merge_data=pd.merge(merge_data,소득가구.rename(columns={"읍면동":'ADM_NM'})[["ADM_NM",'1분위소득가구비율(%)', '2분위소득가구비율(%)', '3분위소득가구비율(%)',
#        '4분위소득가구비율(%)', '5분위소득가구비율(%)']],how='inner')

In [ ]:
# merge_data

소득지출

In [ ]:
소득소비=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\추가데이터\서울시 상권분석서비스(소득소비-행정동).csv",encoding='euc-kr')
소득소비.rename(columns={'행정동_코드_명':'ADM_NM'},inplace=True)
# 월평균소득,지출은 2022년 평균으로 구함

In [ ]:
소득소비["행정동_코드"].value_counts()

행정동_코드
11740690    18
11215780    18
11215760    18
11200520    18
11170685    18
            ..
11500603    18
11470680    18
11470630    18
11470550    18
11740650    18
Name: count, Length: 425, dtype: int64

In [ ]:
i1=소득소비["기준_년분기_코드"]==20221
i2=소득소비["기준_년분기_코드"]==20222
i3=소득소비["기준_년분기_코드"]==20223
i4=소득소비["기준_년분기_코드"]==20224
temp=인구밀도.dropna()
temp.rename(columns={"행정구역코드":"행정동코드","행정동":'ADM_NM',"구":"GU_NAME"})
소득소비=pd.merge(소득소비,temp.rename(columns={"행정구역코드":"행정동코드","행정동":'ADM_NM',"구":"GU_NAME"}))

In [ ]:
인구밀도

,index,행정구역코드,구,행정동,면적 (k㎡),계 (명),인구밀도(명/km²)
0,종로구사직동,1.111053e+09,종로구,사직동,1.23,9371,7618.699187
1,종로구삼청동,1.111054e+09,종로구,삼청동,1.49,2616,1755.704698
2,종로구부암동,1.111055e+09,종로구,부암동,2.27,9460,4167.400881
3,종로구평창동,1.111056e+09,종로구,평창동,8.87,17743,2000.338219
4,종로구무악동,1.111057e+09,종로구,무악동,0.36,8045,22347.222220
...,...,...,...,...,...,...,...
421,강동구둔촌1동,1.174069e+09,강동구,둔촌1동,0.92,74,80.434783
422,강동구둔촌2동,1.174070e+09,강동구,둔촌2동,1.56,24212,15520.512820
423,강동구강일동,1.174052e+09,강동구,강일동,2.26,33082,14638.053100
424,강동구상일1동,1.174052e+09,강동구,상일1동,1.82,39435,21667.582420


In [ ]:
t1=pd.DataFrame(소득소비.groupby(["행정동_코드"])[["월_평균_소득_금액","지출_총금액"]].mean()).reset_index()
t2=인구밀도[["행정구역코드","구","행정동"]].rename(columns={"행정구역코드":'행정동_코드',"구":"GU_NAME","행정동":"ADM_NM"})

In [ ]:
t2.dropna(inplace=True)
t2["행정동_코드"]=t2["행정동_코드"].astype("int")
t2["행정동_코드"]=t2["행정동_코드"].apply(lambda x:int(x/100))

In [ ]:
소득지출=pd.merge(t1,t2,how='inner')
소득지출.rename(columns={"월_평균_소득_금액":'월평균소득',"지출_총금액":"분기평균총지출"},inplace=True)

In [ ]:
merge_data=pd.merge(merge_data,소득지출,how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,...,제방,수문,암거,병원,경찰,소방서,노후건물개수,행정동_코드,월평균소득,분기평균총지출
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,...,3,0,95,75,3,0,485,11110530,5.041217e+06,4.608418e+10
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,...,0,0,43,11,1,0,344,11110540,3.497460e+06,2.433650e+09
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,...,40,3,40,4,1,1,972,11110550,3.662519e+06,1.315638e+09
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,...,83,20,171,15,1,0,1331,11110560,4.546270e+06,2.483992e+09
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,...,0,0,10,6,0,0,101,11110570,5.134728e+06,1.953738e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,...,0,0,17,80,0,0,425,11740660,2.922979e+06,6.422163e+09
403,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,...,4,0,21,103,0,1,892,11740685,2.870842e+06,1.120701e+10
404,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,...,0,0,9,1,0,0,11,11740690,3.596221e+06,2.439333e+07
405,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,...,19,1,68,28,2,0,574,11740700,3.445921e+06,3.795775e+09


사회적약자

In [ ]:
사회적약자=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\장애인노인비율.xlsx")
사회적약자["GU_NAME"].fillna(method="ffill",inplace=True)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_18544\3021968630.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  사회적약자["GU_NAME"].fillna(method="ffill",inplace=True)


In [ ]:
merge_data=pd.merge(merge_data,사회적약자.drop(["전체인구","고령자","장애인"],axis=1),how='inner')

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,...,암거,병원,경찰,소방서,노후건물개수,행정동_코드,월평균소득,분기평균총지출,고령자비율,장애인비율
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,...,95,75,3,0,485,11110530,5.041217e+06,4.608418e+10,0.193266,0.028541
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,...,43,11,1,0,344,11110540,3.497460e+06,2.433650e+09,0.236942,0.041635
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,...,40,4,1,1,972,11110550,3.662519e+06,1.315638e+09,0.190227,0.032613
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,...,171,15,1,0,1331,11110560,4.546270e+06,2.483992e+09,0.195711,0.027495
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,...,10,6,0,0,101,11110570,5.134728e+06,1.953738e+09,0.183308,0.040363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,...,17,80,0,0,425,11740660,2.922979e+06,6.422163e+09,0.177626,0.000130
403,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,...,21,103,0,1,892,11740685,2.870842e+06,1.120701e+10,0.186847,0.036607
404,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,...,9,1,0,0,11,11740690,3.596221e+06,2.439333e+07,0.150685,12.671233
405,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,...,68,28,2,0,574,11740700,3.445921e+06,3.795775e+09,0.175063,0.069696


반지하

In [ ]:
반지하=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\행정동별 지하 수(행정동코드.시군구구추가).xlsx")
반지하["시군구"].fillna(method="ffill",inplace=True)
반지하.rename(columns={"시군구":"GU_NAME","행정동":"ADM_NM"},inplace=True)
반지하.drop(['시군구.1'],axis=1,inplace=True)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_18544\3816143617.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  반지하["시군구"].fillna(method="ffill",inplace=True)


In [ ]:
반지하

,GU_NAME,ADM_NM,행정동코드,면적,지하건물
0,종로구,사직동,1101053,1.23,558
1,종로구,삼청동,1101054,1.49,455
2,종로구,부암동,1101055,2.27,813
3,종로구,평창동,1101056,8.87,1797
4,종로구,무악동,1101057,0.36,58
...,...,...,...,...,...
421,강동구,천호2동,1125073,1.57,1606
422,강동구,길동,1125074,1.61,1319
423,강동구,강일동,1125075,2.26,56
424,강동구,상일1동,1125076,1.82,352


In [ ]:
merge_data=pd.merge(merge_data,반지하)

In [ ]:
merge_data

,GU_NAME,ADM_NM,동면적(k㎡),계 (명),인구밀도(명/km²),동/구면적,면적당시우량,침수횟수,경사도,양호비율,...,소방서,노후건물개수,행정동_코드,월평균소득,분기평균총지출,고령자비율,장애인비율,행정동코드,면적,지하건물
0,종로구,사직동,1.23,9371,7618.699187,0.051443,7.36802,0.0,0.153750,0.245283,...,0,485,11110530,5.041217e+06,4.608418e+10,0.193266,0.028541,1101053,1.23,558
1,종로구,삼청동,1.49,2616,1755.704698,0.062317,7.36802,0.0,0.250000,0.832402,...,0,344,11110540,3.497460e+06,2.433650e+09,0.236942,0.041635,1101054,1.49,455
2,종로구,부암동,2.27,9460,4167.400881,0.094939,7.36802,2.0,0.314583,0.906250,...,1,972,11110550,3.662519e+06,1.315638e+09,0.190227,0.032613,1101055,2.27,813
3,종로구,평창동,8.87,17743,2000.338219,0.370974,7.36802,2.0,0.229615,0.974132,...,0,1331,11110560,4.546270e+06,2.483992e+09,0.195711,0.027495,1101056,8.87,1797
4,종로구,무악동,0.36,8045,22347.222220,0.015056,7.36802,0.0,0.346667,1.000000,...,0,101,11110570,5.134728e+06,1.953738e+09,0.183308,0.040363,1101057,0.36,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,강동구,성내3동,0.71,22984,32371.830990,0.028874,8.79600,12.0,0.072143,0.633929,...,0,425,11740660,2.922979e+06,6.422163e+09,0.177626,0.000130,1125067,0.71,788
403,강동구,길동,1.61,45225,28090.062110,0.065474,8.79600,9.0,0.176154,0.608911,...,1,892,11740685,2.870842e+06,1.120701e+10,0.186847,0.036607,1125074,1.61,1319
404,강동구,둔촌1동,0.92,74,80.434783,0.037414,8.79600,3.0,0.151786,0.738636,...,0,11,11740690,3.596221e+06,2.439333e+07,0.150685,12.671233,1125070,0.92,6
405,강동구,둔촌2동,1.56,24212,15520.512820,0.063440,8.79600,10.0,0.170909,0.797872,...,0,574,11740700,3.445921e+06,3.795775e+09,0.175063,0.069696,1125071,1.56,714


In [ ]:
merge_data.to_excel("./빗물펌프장추가필요.xlsx")

Index(['GU_NAME', 'ADM_NM', '동면적(k㎡)', '계 (명)', '인구밀도(명/km²)', '동/구면적',
       '면적당시우량', '침수횟수', '경사도', '양호비율', '도로_건물_면적_비율', '녹지_면적_비율', '제방', '수문',
       '암거', '병원', '경찰', '소방서', '노후건물개수', '행정동_코드', '월평균소득', '분기평균총지출',
       '고령자비율', '장애인비율', '행정동코드', '면적', '지하건물'],
      dtype='object')